# CNN model 
This code below is all about CNN model to classify MNIST datasets. It is demonstrating a shallow CNN model code simulation.

### Header

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init as init

### Select between cuda or cpu

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: {}'.format(device))

device: cuda


### Give a random seed

In [ ]:
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

### Select hyper parameters

In [ ]:
learning_rate = 1e-3
training_epochs = 15
batch_size = 100

### Dataset

In [ ]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True,  transform = transforms.ToTensor(),download=True)
mnist_test  = dsets.MNIST(root='MNIST_data/', train=False, transform = transforms.ToTensor(),download=True)


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
dataloader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle = True, drop_last = True)

### Model

when we use `out.size(0)`, it returns first element of torch.Tensor's size. It can be replaced by `out.size()[0]` The code for usage of this `out.size(0)` is below.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride = 2)
        ).to(device)

        self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        ).to(device)
        
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True).to(device)
        init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
model = CNN()

### Select optimizer and cost function

In [ ]:
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
iteration = len(dataloader)
print('iteration: {}'.format(iteration))

iteration: 600


### Training

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for index, (X, Y) in enumerate(dataloader):
        X = X.to(device)
        Y = Y.to(device)
        hypothesis = model(X)
        cost = loss_fn(hypothesis, Y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / iteration
        
    print('[Epoch: {:4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.227677807
[Epoch:    2] cost = 0.0609042607
[Epoch:    3] cost = 0.0441043898
[Epoch:    4] cost = 0.0354618728
[Epoch:    5] cost = 0.0294378791
[Epoch:    6] cost = 0.0244312752
[Epoch:    7] cost = 0.0203469805
[Epoch:    8] cost = 0.0174251031
[Epoch:    9] cost = 0.0144119812
[Epoch:   10] cost = 0.0125781149
[Epoch:   11] cost = 0.00969043188
[Epoch:   12] cost = 0.00987839233
[Epoch:   13] cost = 0.00789897889
[Epoch:   14] cost = 0.00652727345
[Epoch:   15] cost = 0.00643697381


### Validation

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, dim = 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: {}'.format(accuracy.item()))

Accuracy: 0.9782999753952026


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
